In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pylab as pl
#from pandas_profiling import ProfileReport
from sklearn import preprocessing
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import seaborn as sns
from pandas.io.json import json_normalize 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import svm
import datetime
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import math
%matplotlib inline

In [2]:
cancer=pd.read_csv('elsa_cervical_data.csv')

In [3]:
cancer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1847 entries, 0 to 1846
Data columns (total 84 columns):
Unnamed: 0                            1847 non-null int64
age                                   1847 non-null object
alcohol                               1847 non-null object
cervicalCancer                        1847 non-null bool
cervicalCancerType                    1847 non-null object
children                              1847 non-null object
date                                  1847 non-null object
existing_conditions                   1426 non-null object
family_history                        1847 non-null object
gender                                1847 non-null object
height                                1847 non-null object
region                                1845 non-null object
smokes                                1847 non-null object
symptomDuration                       1320 non-null object
symptoms                              0 non-null float64
temperature  

In [134]:
#cancer['pain during sexual intercourse'].value_counts()
#cancer['back pain'].isna().sum()
#cancer.head(300)
#cancer.cervicalCancer.where(cancer['prolonged vaginal bleeding'] == True).value_counts()
pd.crosstab(cancer.cervicalCancer, cancer['existing_conditions'])

existing_conditions,"['diabetes', 'none']",['diabetes'],"['hypertension', 'diabetes', 'none']","['hypertension', 'diabetes', 'tb', 'none']","['hypertension', 'diabetes']","['hypertension', 'none']",['hypertension'],['none'],"['other-cancers', 'none']",['other-cancers']
cervicalCancer,,,,,,,,,,
False,0,0,0,0,1,0,0,636,0,0
True,1,2,2,1,2,2,5,771,2,1


In [4]:
cancer.describe()

,Unnamed: 0,symptoms,tests,baso,bilirubin,creatinine,eos,epithelial,hb,lymphocyte,mch,mcv,neutrophil,ph,plt,protein,rbc,urea,uric-acid,wbc,ast,random-blood-glucose-rbg,lower lumbs swelling,pervic pain,polpitations
count,1847.000000,0.0,0.0,1402.000000,207.000000,282.000000,1403.000000,1.0,1408.000000,1415.000000,1412.000000,1415.000000,1394.000000,2.000000,1409.000000,155.000000,1412.000000,293.000000,178.000000,1411.000000,20.000000,0.0,3.0,3.0,3.0
mean,923.000000,NaN,NaN,3.792951,5.085797,181.073014,9.989090,-1.0,19.298033,21.586947,23.997805,75.398304,65.561628,3.000000,347.223322,85.103871,4.204703,6.284403,164.515038,8.390057,45.740000,NaN,1.0,1.0,1.0
std,533.327292,NaN,NaN,43.858285,9.742102,421.004548,181.987612,NaN,34.639240,68.760127,9.815792,37.716637,18.462072,2.828427,131.973273,44.265085,8.582868,8.204584,153.889281,14.324152,76.589082,NaN,0.0,0.0,0.0
min,0.000000,NaN,NaN,0.000000,-0.480000,0.170000,0.000000,-1.0,1.300000,0.480000,8.700000,3.600000,0.350000,1.000000,2.730000,1.840000,0.090000,0.660000,0.172700,1.570000,10.490000,NaN,1.0,1.0,1.0
25%,461.500000,NaN,NaN,0.200000,0.285000,66.567500,1.540000,-1.0,7.700000,12.950000,20.200000,66.600000,59.000000,2.000000,262.000000,70.945000,3.100000,2.720000,6.385000,4.420000,18.000000,NaN,1.0,1.0,1.0
50%,923.000000,NaN,NaN,0.300000,2.350000,85.280000,2.800000,-1.0,9.700000,13.500000,23.450000,74.400000,71.000000,3.000000,346.000000,80.360000,3.710000,3.570000,179.650000,7.780000,21.105000,NaN,1.0,1.0,1.0
75%,1384.500000,NaN,NaN,0.400000,6.110000,115.952500,4.700000,-1.0,11.200000,23.950000,27.100000,79.150000,78.000000,4.000000,449.000000,91.525000,4.700000,5.400000,271.452500,10.285000,41.350000,NaN,1.0,1.0,1.0
max,1846.000000,NaN,NaN,868.000000,95.030000,4989.310000,6767.000000,-1.0,710.000000,2206.000000,337.000000,988.600000,95.800000,5.000000,1142.000000,322.040000,324.000000,56.000000,631.140000,512.000000,363.500000,NaN,1.0,1.0,1.0


In [4]:
#profile = ProfileReport(cancer)
#profile

In [5]:
cancer.drop(['Unnamed: 0','random-blood-glucose-rbg','date','height','symptomDuration','symptoms','tests',
             'weight','alt','ph','ast','bilirubin','epithelial','protein','urea','uric-acid','hpv','mri',
'weight loss','nausea','pain during sexual intercourse','vomiting',
'difficulty passing stool','fever','cough','difficulty micturation','chest pain',
'lower back pain','frequent micturation','lower limbs swelling','fatigue','swelling','rectal bleeding','constipation',
'difficulty in breathing','occasional painful defecation',
'blood in urine','headache','palpitations','vulva itching','vaginal pain',
'spot bleeding','metromenorrhagia','lower lumbs swelling','pervic pain','polpitations','creatinine'], axis=1, inplace=True)

In [6]:
 cancer['cervicalCancer'].value_counts()

True     999
False    848
Name: cervicalCancer, dtype: int64

In [7]:
# cancer.loc[cancer['cervicalCancer'] == False]

In [8]:
# cancer.dtypes

In [9]:
# #none are highly frequency
# # cancer['existing_conditions'].value_counts()
# cancer.fillna(cancer.mean(),inplace=True)

In [10]:
cancer['age'].replace('    38', 38, inplace = True) 

In [11]:
cancer['age'].replace('?', np.nan,inplace = True)
#drop height column
#cancer['symptoms'].value_counts()

In [12]:
cancer['age'].replace('4637',46, inplace = True)

In [13]:
cancer['age'].replace(' 52', 52, inplace = True)

In [14]:
cancer['age'].replace('A- D', np.nan, inplace = True)
 

In [15]:
# cancer['age'].unique()

In [16]:
# cancer['age'].value_counts()

In [17]:
cancer['age']=cancer['age'].astype(float)
cancer['age'].mean()

52.69885433715221

In [18]:
cancer['age'].replace(np.nan,52.68,inplace=True)

In [19]:
cancer['age'].isnull().sum()

0

In [20]:
cancer['cervicalCancer'].replace(True,1,inplace=True)

In [21]:
cancer['cervicalCancer'].replace(False,0,inplace=True)

In [22]:
cancer['cervicalCancer'].isnull().sum()

0

In [23]:
# cancer['children'].value_counts()

In [24]:
cancer['children'].replace('unknown',np.nan,inplace=True)
cancer['children'].replace('7+',7,inplace=True)
cancer['children'].isnull().sum()

630

In [25]:
cancer['children']=cancer['children'].astype(float)
cancer['children'].mean()

5.531635168447001

In [26]:
cancer['children'].replace(np.nan,4.75,inplace=True)
cancer['children'].isnull().sum()

0

In [27]:
cancer['children'].replace(7.00,'7+',inplace=True)

In [28]:
# cancer['cervicalCancerType'].unique()

In [29]:
# cancer['cervicalCancerType'].value_counts()

In [30]:
cancer['cervicalCancerType'].replace('?','squamous',inplace=True)


In [31]:
cancer['cervicalCancerType'].isnull().sum()

0

In [32]:
cancer['existing_conditions'].isnull().sum()

421

In [33]:
# cancer['existing_conditions'].value_counts()

In [34]:
cancer['existing_conditions'].replace(np.nan,"['none']",inplace=True)
# cancer['existing_conditions'].value_counts()

In [35]:
# cancer['region'].value_counts()

In [36]:
cancer['region'].replace(np.nan,'Dar Es Salaam',inplace=True)

In [37]:
cancer['smokes'].unique()

array(['no', 'unknown', 'yes'], dtype=object)

In [38]:
cancer['temperature'].replace(' ',np.nan,inplace=True)
cancer['temperature'].replace('?',np.nan,inplace=True)

In [39]:
cancer['temperature']=cancer['temperature'].astype(float)
#cancer['temperature'].unique()
#cancer['temperature'].mean()

In [40]:
cancer['temperature'].replace(np.nan,36.23,inplace=True)

In [41]:
# cancer['creatinine'].isnull().sum()

In [42]:
# cancer.loc[cancer['cervicalCancer'] == False]

In [43]:
cancer['hiv'].fillna(False,inplace=True)

In [44]:
cancer['temperature'].replace('37.  ',37, inplace=True)

In [45]:
cancer['prolonged vaginal bleeding'].fillna(False,inplace=True)

In [46]:
cancer['back pain'].fillna(False,inplace=True)

In [47]:
cancer['abdominal pain'].fillna(False,inplace=True)

In [48]:
cancer['vaginal discharge'].fillna(False,inplace=True)

In [49]:
cancer['abnormal bleeding'].fillna(False,inplace=True)

In [50]:
cancer['lower abdominal pain'].fillna(False,inplace=True)

In [51]:
cancer['yellowish vaginal discharge'].fillna(False,inplace=True)

In [52]:
cancer['post menopausal bleeding'].fillna(False,inplace=True)

In [53]:
cancer['prolonged vaginal watery discharge'].fillna(False,inplace=True)

In [54]:
cancer['pelvic pain'].fillna(False,inplace=True)

In [55]:
cancer['waist pain'].fillna(False,inplace=True)

In [56]:
cancer['foul smelling discharge'].fillna(False,inplace=True)

In [57]:
cancer['post coital bleeding'].fillna(False,inplace=True)

In [58]:
# data_report = ProfileReport(cancer)

In [59]:
# import seaborn as sns

# plt.figure(figsize=(10,9))
# sns.heatmap(cancer.corr(),annot=True,cmap='winter')

In [60]:
cancer['baso'].replace(np.nan,cancer['baso'].mean(),inplace=True)

In [61]:
cancer['eos'].replace(np.nan,cancer['eos'].mean(),inplace=True)

In [62]:
cancer['hb'].replace(np.nan,cancer['hb'].mean(),inplace=True)

In [63]:
cancer['mch'].replace(np.nan,cancer['mch'].mean(),inplace=True)

In [64]:
cancer['neutrophil'].replace(np.nan,cancer['neutrophil'].mean(),inplace=True)

In [65]:
cancer['plt'].replace(np.nan,cancer['plt'].mean(),inplace=True)

In [66]:
cancer['rbc'].replace(np.nan,cancer['rbc'].mean(),inplace=True)

In [67]:
cancer['wbc'].replace(np.nan,cancer['wbc'].mean(),inplace=True)

In [68]:
cancer['chest-x-ray'].replace(np.nan,False,inplace=True)

In [69]:
cancer['ultrasound'].replace(np.nan,False,inplace=True)

In [70]:
cancer['mcv'].replace(np.nan,cancer['mcv'].mean(),inplace=True)

In [71]:
cancer['lymphocyte'].replace(np.nan,cancer['lymphocyte'].mean(),inplace=True)

In [72]:
# cancer['lymphocyte'].value_counts()

In [73]:
cancer['chest-x-ray'].unique()

array([False,  True])

In [74]:
data_report = ProfileReport(cancer)

NameError: name 'ProfileReport' is not defined

In [75]:
data_report

NameError: name 'data_report' is not defined

In [76]:
# cancer=pd.get_dumies(cancer)
cancer = pd.get_dummies(cancer)

In [77]:
# cancer.loc[cancer['cervicalCancer'] == False]

In [78]:
cancer['cervicalCancer'].value_counts()

1.0    999
0.0    848
Name: cervicalCancer, dtype: int64

In [79]:
feature= cancer.drop(columns='cervicalCancer')
target= cancer['cervicalCancer'].copy()

In [80]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.7, random_state=10)

In [83]:
print(feature.head())

    age  temperature   baso    eos    hb  lymphocyte        mch    mcv  \
0  40.0        40.00  10.00  10.00  10.0        10.0  23.997805   10.0   
1  36.0        36.23   0.30   2.20   8.8        34.9  19.000000   74.4   
2  40.0        36.23   0.10   3.50  10.6        43.6  19.700000   74.7   
3  48.0        36.23   0.10   0.90   6.2        15.3  22.000000   75.0   
4  55.0        36.23   0.44   1.12  12.6        12.7  32.900000  108.0   

   neutrophil    plt    rbc    wbc  chest-x-ray    hiv  ultrasound  \
0   10.000000   10.0  10.00  10.00        False  False        True   
1   65.561628  303.0   4.61   5.26        False  False       False   
2   65.561628  235.0   5.37   7.16        False  False       False   
3   65.561628  374.0   2.79  15.21        False  False        True   
4   77.700000  193.0   3.56   6.58        False   True        True   

   prolonged vaginal bleeding  back pain  abdominal pain  vaginal discharge  \
0                        True      False            Tru

In [84]:
from sklearn.metrics import confusion_matrix


# Evaluation

In [ ]:

#models = []
# models.append(( KNeighborsClassifier(3)))
# models.append(( RandomForestClassifier(n_estimators=100)))
# models.append(( SVC(gamma=3, C=1)))

# # Evaluations
# df_out = pd.DataFrame({ 'model':[],'Train_score':[], 
#                     'Test_score':[]}) 
# modlist = ['KNeighborsClassifier','RandomForestClassifier','SVC']
# poi=0
# i=0
# for item in models:
#     mymodel=item
#     mymodel.fit(X_train,y_train)
#     training=mymodel.score(X_train,y_train)*100
#     testing=mymodel.score(X_test,y_test)*100
#     y_pred=mymodel.predict(X_test)
#     print(modlist[i])
#     print('Train Score:',training)
#     print('Test Score:',testing)
#     print('Classification Report:' +'\n'+'\n'+ classification_report(y_test,y_pred))
#     print('CONFUSION MATRIX'+'\n',confusion_matrix(y_test, y_pred))
#     print('\n'+ '\n')
#    # Add a new row at index position 2 with values provided in list
#    #df_out.loc[poi] = [modlist[poi],training, testing]
#     poi+=1
#     i+=1
  

## Random Forest Classifier

In [85]:
model= RandomForestClassifier(n_estimators=100)
model.fit(X_train,y_train)
training=model.score(X_train,y_train)*100
testing=model.score(X_test,y_test)*100
ypred=model.predict(X_test)

In [108]:
#Predicting a single record
test_idx=np.arange(X_test.shape[0])==36

npred = model.predict(X_test[test_idx])
print(npred)
print(y_test[test_idx])
X_test[test_idx]


[1.]
312    1.0
Name: cervicalCancer, dtype: float64


,age,temperature,baso,eos,hb,lymphocyte,mch,mcv,neutrophil,plt,rbc,wbc,chest-x-ray,hiv,ultrasound,prolonged vaginal bleeding,back pain,abdominal pain,vaginal discharge,abnormal bleeding,lower abdominal pain,post menopausal bleeding,prolonged vaginal watery discharge,waist pain,pelvic pain,yellowish vaginal discharge,foul smelling discharge,post coital bleeding,alcohol_no,alcohol_unknown,alcohol_yes,cervicalCancerType_adenocarcinoma,cervicalCancerType_differentiated,cervicalCancerType_metastatic,cervicalCancerType_small-cell,cervicalCancerType_squamous,cervicalCancerType_undifferentiated,children_0.0,children_1.0,children_2.0,children_3.0,children_4.0,children_4.75,children_5.0,children_6.0,children_7+,"existing_conditions_['diabetes', 'none']",existing_conditions_['diabetes'],"existing_conditions_['hypertension', 'diabetes', 'none']","existing_conditions_['hypertension', 'diabetes', 'tb', 'none']","existing_conditions_['hypertension', 'diabetes']","existing_conditions_['hypertension', 'none']",existing_conditions_['hypertension'],existing_conditions_['none'],"existing_conditions_['other-cancers', 'none']",existing_conditions_['other-cancers'],family_history_no,family_history_unknown,family_history_yes,gender_female,region_Arusha,region_Babati,region_Bukoba,region_Dar Es Salaam,region_Dodoma,region_Geita,region_Handeni,region_Iringa,region_Kagera,region_Kahama,region_Kasulu,region_Katavi,region_Kibaha,region_Kigoma,region_Kilimanjaro,region_Lindi,region_Manyara,region_Mara,region_Mbeya,region_Morogoro,region_Moshi,region_Mpanda,region_Mtwara,region_Musoma,region_Mwanza,region_Njombe,region_Other,region_Pwani,region_Rukwa,region_Ruvuma,region_Shinyanga,region_Simiyu,region_Singida,region_Sumbawanga,region_Tabora,region_Tanga,region_Zanzibar,smokes_no,smokes_unknown,smokes_yes
312,62.0,36.0,0.3,4.6,7.3,20.5,20.2,73.8,66.3,231.0,3.59,7.82,False,False,False,True,False,False,True,True,False,False,False,False,False,False,True,False,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [86]:
print('Train Score:',training)
print('Test Score:',testing)
print('Classification Report:' +'\n'+'\n'+ classification_report(y_test,ypred))
print('CONFUSION MATRIX'+'\n',confusion_matrix(y_test, ypred))

Train Score: 100.0
Test Score: 99.61330239752513
Classification Report:

              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00       602
         1.0       1.00      0.99      1.00       691

   micro avg       1.00      1.00      1.00      1293
   macro avg       1.00      1.00      1.00      1293
weighted avg       1.00      1.00      1.00      1293

CONFUSION MATRIX
 [[601   1]
 [  4 687]]


In [ ]:
# d4 = {'tcc':y_test, 'cervicalCancer': ypred}
# test_predictions4 = pd.DataFrame(data=d4)
# test_predictions4 = test_predictions4[['tcc','cervicalCancer']]
# #test_predictions4

## KnNeighbors Classifier

In [ ]:
knn= KNeighborsClassifier(3)
knn.fit(X_train,y_train)
trainin=knn.score(X_train,y_train)*100
testin=knn.score(X_test,y_test)*100
y_pred1=knn.predict(X_test)

In [ ]:
print('Train Score:',trainin)
print('Test Score:',testin)
print('Classification Report:' +'\n'+'\n'+ classification_report(y_test,y_pred1))
print('CONFUSION MATRIX'+'\n',confusion_matrix(y_test, y_pred1))

In [ ]:
#  d = {'tcc':y_test, 'cervicalCancer': y_pred1}
# test_predictions = pd.DataFrame(data=d)
# test_predictions = test_predictions4[['tcc','cervicalCancer']]
# test_predictions.tail()